Video Classification Model : https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/

## STEP 1:  import the required libraries

In [29]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2
from glob import glob
import os

## STEP 2: Read each frame and their corresponding tag

In [21]:
train = pd.read_csv('train_new.csv')
train.head()

,image,class
0,A_OffSide_shot1.mp4_frame0.jpg,OffSide
1,A_OffSide_shot1.mp4_frame1.jpg,OffSide
2,A_OffSide_shot1.mp4_frame10.jpg,OffSide
3,A_OffSide_shot1.mp4_frame100.jpg,OffSide
4,A_OffSide_shot1.mp4_frame101.jpg,OffSide


## STEP 3: Read the frames and then store them as a NumPy array

In [3]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|████████████████████████████████████████████████████████████████████████████| 10991/10991 [04:58<00:00, 36.85it/s]


(10991, 224, 224, 3)

## STEP 4: Create the validation set

In [4]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [32]:
# separating the target
y_df = y.to_frame()

## STEP 5: Create 4 different columns in the target, one for each category

In [5]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

## STEP 6: Use the VGG-16 pre-trained model to create the base model

Since we do not have a very large dataset, creating a model from scratch might not work well. So, we will use a pre-trained model and take its learnings to solve our problem

In [6]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

52994048/58889256 [=========================>....] - ETA: 28:1 - ETA: 11:2 - ETA: 6:3 - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 57s - ETA: 48 - ETA: 50 - ETA: 46 - ETA: 46 - ETA: 40 - ETA: 42 - ETA: 37 - ETA: 39 - ETA: 36 - ETA: 37 - ETA: 34 - ETA: 36 - ETA: 36 - ETA: 34 - ETA: 33 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 31 - ETA: 32 - ETA: 31 - ETA: 31 - ETA: 30 - ETA: 31 - ETA: 30 - ETA: 31 - ETA: 30 - ETA: 29 - ETA: 30 - ETA: 29 - ETA: 30 - ETA: 31 - ETA: 29 - ETA: 30 - ETA: 29 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 31 - ETA: 30 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 34 - ETA: 33 - ETA: 33 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - E

58892288/58889256 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA

## STEP 7: Extract features from this pre-trained model for our training and validation images

In [7]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(8792, 7, 7, 512)

In [8]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(2199, 7, 7, 512)

## STEP 8: reshape the images into a single dimension

In [9]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(8792, 7*7*512)
X_test = X_test.reshape(2199, 7*7*512)

## STEP 9: Normalize the pixel values (between 0 and 1, helps the model to converge faster)

In [10]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

## STEP 10: Create the architecture of the model

In [11]:
# shape of images
X_train.shape

(8792, 25088)

In [12]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## STEP 11: Train our model using the training frames and validate using validation frames

In [13]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [14]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
__________

In [16]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 8792 samples, validate on 2199 samples
Epoch 1/200
8792/8792 [==============================] - ETA: 2:56 - loss: 1.3962 - acc: 0.265 - ETA: 1:40 - loss: 1.3961 - acc: 0.316 - ETA: 1:14 - loss: 1.3638 - acc: 0.359 - ETA: 1:00 - loss: 1.3853 - acc: 0.396 - ETA: 52s - loss: 1.3628 - acc: 0.423 - ETA: 46s - loss: 1.3390 - acc: 0.43 - ETA: 43s - loss: 1.3309 - acc: 0.43 - ETA: 40s - loss: 1.3252 - acc: 0.43 - ETA: 38s - loss: 1.3244 - acc: 0.43 - ETA: 36s - loss: 1.3193 - acc: 0.43 - ETA: 34s - loss: 1.3129 - acc: 0.44 - ETA: 33s - loss: 1.3011 - acc: 0.45 - ETA: 31s - loss: 1.2944 - acc: 0.45 - ETA: 30s - loss: 1.2889 - acc: 0.45 - ETA: 29s - loss: 1.2920 - acc: 0.45 - ETA: 28s - loss: 1.2840 - acc: 0.46 - ETA: 27s - loss: 1.2788 - acc: 0.46 - ETA: 26s - loss: 1.2808 - acc: 0.46 - ETA: 26s - loss: 1.2763 - acc: 0.47 - ETA: 25s - loss: 1.2772 - acc: 0.46 - ETA: 25s - loss: 1.2667 - acc: 0

8792/8792 [==============================] - ETA: 24s - loss: 0.9375 - acc: 0.62 - ETA: 25s - loss: 1.0078 - acc: 0.55 - ETA: 25s - loss: 0.9816 - acc: 0.55 - ETA: 24s - loss: 0.9766 - acc: 0.56 - ETA: 24s - loss: 0.9872 - acc: 0.54 - ETA: 23s - loss: 0.9833 - acc: 0.54 - ETA: 23s - loss: 0.9647 - acc: 0.55 - ETA: 22s - loss: 0.9612 - acc: 0.55 - ETA: 22s - loss: 0.9621 - acc: 0.55 - ETA: 21s - loss: 0.9658 - acc: 0.55 - ETA: 22s - loss: 0.9656 - acc: 0.54 - ETA: 22s - loss: 0.9597 - acc: 0.55 - ETA: 23s - loss: 0.9580 - acc: 0.55 - ETA: 23s - loss: 0.9478 - acc: 0.56 - ETA: 23s - loss: 0.9468 - acc: 0.56 - ETA: 22s - loss: 0.9452 - acc: 0.56 - ETA: 22s - loss: 0.9410 - acc: 0.56 - ETA: 21s - loss: 0.9372 - acc: 0.56 - ETA: 21s - loss: 0.9404 - acc: 0.56 - ETA: 20s - loss: 0.9385 - acc: 0.57 - ETA: 20s - loss: 0.9404 - acc: 0.56 - ETA: 19s - loss: 0.9418 - acc: 0.56 - ETA: 19s - loss: 0.9374 - acc: 0.57 - ETA: 19s - loss: 0.9365 - acc: 0.57 - ETA: 18s - loss: 0.9350 - acc: 0.57 - ETA: 

8792/8792 [==============================] - ETA: 23s - loss: 0.5097 - acc: 0.80 - ETA: 23s - loss: 0.5920 - acc: 0.76 - ETA: 25s - loss: 0.5913 - acc: 0.75 - ETA: 25s - loss: 0.5439 - acc: 0.78 - ETA: 24s - loss: 0.5416 - acc: 0.77 - ETA: 23s - loss: 0.5297 - acc: 0.77 - ETA: 23s - loss: 0.5317 - acc: 0.77 - ETA: 22s - loss: 0.5251 - acc: 0.78 - ETA: 22s - loss: 0.5309 - acc: 0.78 - ETA: 21s - loss: 0.5266 - acc: 0.78 - ETA: 21s - loss: 0.5271 - acc: 0.78 - ETA: 21s - loss: 0.5367 - acc: 0.78 - ETA: 20s - loss: 0.5312 - acc: 0.78 - ETA: 20s - loss: 0.5335 - acc: 0.78 - ETA: 20s - loss: 0.5371 - acc: 0.77 - ETA: 19s - loss: 0.5324 - acc: 0.77 - ETA: 19s - loss: 0.5293 - acc: 0.77 - ETA: 18s - loss: 0.5336 - acc: 0.77 - ETA: 18s - loss: 0.5293 - acc: 0.77 - ETA: 18s - loss: 0.5280 - acc: 0.77 - ETA: 17s - loss: 0.5292 - acc: 0.77 - ETA: 17s - loss: 0.5264 - acc: 0.78 - ETA: 16s - loss: 0.5291 - acc: 0.77 - ETA: 16s - loss: 0.5320 - acc: 0.77 - ETA: 16s - loss: 0.5329 - acc: 0.77 - ETA: 

8792/8792 [==============================] - ETA: 34s - loss: 0.3812 - acc: 0.81 - ETA: 33s - loss: 0.4290 - acc: 0.78 - ETA: 30s - loss: 0.4003 - acc: 0.79 - ETA: 29s - loss: 0.4084 - acc: 0.79 - ETA: 28s - loss: 0.4299 - acc: 0.78 - ETA: 28s - loss: 0.4289 - acc: 0.78 - ETA: 27s - loss: 0.4223 - acc: 0.78 - ETA: 26s - loss: 0.4278 - acc: 0.78 - ETA: 27s - loss: 0.4229 - acc: 0.78 - ETA: 27s - loss: 0.4238 - acc: 0.79 - ETA: 28s - loss: 0.4295 - acc: 0.79 - ETA: 27s - loss: 0.4348 - acc: 0.78 - ETA: 26s - loss: 0.4347 - acc: 0.79 - ETA: 26s - loss: 0.4424 - acc: 0.78 - ETA: 25s - loss: 0.4432 - acc: 0.78 - ETA: 25s - loss: 0.4436 - acc: 0.78 - ETA: 24s - loss: 0.4427 - acc: 0.79 - ETA: 23s - loss: 0.4418 - acc: 0.79 - ETA: 22s - loss: 0.4397 - acc: 0.79 - ETA: 22s - loss: 0.4451 - acc: 0.79 - ETA: 21s - loss: 0.4494 - acc: 0.78 - ETA: 20s - loss: 0.4482 - acc: 0.78 - ETA: 20s - loss: 0.4500 - acc: 0.78 - ETA: 19s - loss: 0.4521 - acc: 0.78 - ETA: 19s - loss: 0.4499 - acc: 0.78 - ETA: 

8792/8792 [==============================] - ETA: 26s - loss: 0.5041 - acc: 0.75 - ETA: 25s - loss: 0.5086 - acc: 0.75 - ETA: 25s - loss: 0.4923 - acc: 0.76 - ETA: 24s - loss: 0.4542 - acc: 0.77 - ETA: 24s - loss: 0.4678 - acc: 0.77 - ETA: 24s - loss: 0.4846 - acc: 0.76 - ETA: 24s - loss: 0.4691 - acc: 0.78 - ETA: 23s - loss: 0.4681 - acc: 0.78 - ETA: 23s - loss: 0.4701 - acc: 0.78 - ETA: 22s - loss: 0.4698 - acc: 0.77 - ETA: 21s - loss: 0.4667 - acc: 0.78 - ETA: 21s - loss: 0.4637 - acc: 0.78 - ETA: 20s - loss: 0.4699 - acc: 0.78 - ETA: 20s - loss: 0.4684 - acc: 0.78 - ETA: 19s - loss: 0.4655 - acc: 0.78 - ETA: 19s - loss: 0.4621 - acc: 0.79 - ETA: 18s - loss: 0.4605 - acc: 0.79 - ETA: 18s - loss: 0.4587 - acc: 0.79 - ETA: 18s - loss: 0.4600 - acc: 0.79 - ETA: 17s - loss: 0.4587 - acc: 0.79 - ETA: 17s - loss: 0.4553 - acc: 0.79 - ETA: 16s - loss: 0.4526 - acc: 0.79 - ETA: 16s - loss: 0.4480 - acc: 0.79 - ETA: 16s - loss: 0.4496 - acc: 0.79 - ETA: 15s - loss: 0.4550 - acc: 0.79 - ETA: 

...

8792/8792 [==============================] - ETA: 24s - loss: 0.2514 - acc: 0.91 - ETA: 26s - loss: 0.2574 - acc: 0.89 - ETA: 25s - loss: 0.2796 - acc: 0.88 - ETA: 26s - loss: 0.2670 - acc: 0.89 - ETA: 26s - loss: 0.2924 - acc: 0.88 - ETA: 26s - loss: 0.2994 - acc: 0.88 - ETA: 25s - loss: 0.3124 - acc: 0.88 - ETA: 25s - loss: 0.3100 - acc: 0.87 - ETA: 24s - loss: 0.3270 - acc: 0.87 - ETA: 24s - loss: 0.3368 - acc: 0.87 - ETA: 23s - loss: 0.3284 - acc: 0.87 - ETA: 24s - loss: 0.3289 - acc: 0.87 - ETA: 24s - loss: 0.3255 - acc: 0.87 - ETA: 24s - loss: 0.3199 - acc: 0.87 - ETA: 24s - loss: 0.3179 - acc: 0.87 - ETA: 24s - loss: 0.3141 - acc: 0.87 - ETA: 24s - loss: 0.3152 - acc: 0.87 - ETA: 23s - loss: 0.3123 - acc: 0.87 - ETA: 23s - loss: 0.3120 - acc: 0.87 - ETA: 22s - loss: 0.3130 - acc: 0.87 - ETA: 22s - loss: 0.3129 - acc: 0.87 - ETA: 21s - loss: 0.3162 - acc: 0.87 - ETA: 21s - loss: 0.3206 - acc: 0.87 - ETA: 21s - loss: 0.3232 - acc: 0.87 - ETA: 20s - loss: 0.3231 - acc: 0.87 - ETA: 

8792/8792 [==============================] - ETA: 24s - loss: 0.3342 - acc: 0.85 - ETA: 23s - loss: 0.3176 - acc: 0.87 - ETA: 23s - loss: 0.3081 - acc: 0.87 - ETA: 23s - loss: 0.2841 - acc: 0.89 - ETA: 22s - loss: 0.2731 - acc: 0.89 - ETA: 22s - loss: 0.2891 - acc: 0.89 - ETA: 22s - loss: 0.2862 - acc: 0.89 - ETA: 22s - loss: 0.2792 - acc: 0.89 - ETA: 22s - loss: 0.2843 - acc: 0.89 - ETA: 22s - loss: 0.2788 - acc: 0.89 - ETA: 22s - loss: 0.2786 - acc: 0.89 - ETA: 22s - loss: 0.2737 - acc: 0.89 - ETA: 22s - loss: 0.2699 - acc: 0.89 - ETA: 22s - loss: 0.2699 - acc: 0.89 - ETA: 22s - loss: 0.2693 - acc: 0.89 - ETA: 22s - loss: 0.2699 - acc: 0.89 - ETA: 21s - loss: 0.2694 - acc: 0.89 - ETA: 21s - loss: 0.2645 - acc: 0.89 - ETA: 21s - loss: 0.2640 - acc: 0.89 - ETA: 20s - loss: 0.2672 - acc: 0.89 - ETA: 20s - loss: 0.2721 - acc: 0.89 - ETA: 19s - loss: 0.2743 - acc: 0.89 - ETA: 19s - loss: 0.2718 - acc: 0.89 - ETA: 18s - loss: 0.2709 - acc: 0.89 - ETA: 18s - loss: 0.2670 - acc: 0.89 - ETA: 

8792/8792 [==============================] - ETA: 24s - loss: 0.2725 - acc: 0.89 - ETA: 23s - loss: 0.2713 - acc: 0.90 - ETA: 23s - loss: 0.2889 - acc: 0.88 - ETA: 23s - loss: 0.2955 - acc: 0.88 - ETA: 22s - loss: 0.3100 - acc: 0.87 - ETA: 22s - loss: 0.2984 - acc: 0.88 - ETA: 21s - loss: 0.2938 - acc: 0.87 - ETA: 21s - loss: 0.2968 - acc: 0.87 - ETA: 21s - loss: 0.2969 - acc: 0.88 - ETA: 20s - loss: 0.2834 - acc: 0.88 - ETA: 20s - loss: 0.2833 - acc: 0.88 - ETA: 20s - loss: 0.2793 - acc: 0.88 - ETA: 19s - loss: 0.2770 - acc: 0.88 - ETA: 19s - loss: 0.2696 - acc: 0.88 - ETA: 19s - loss: 0.2724 - acc: 0.88 - ETA: 18s - loss: 0.2735 - acc: 0.88 - ETA: 18s - loss: 0.2723 - acc: 0.88 - ETA: 18s - loss: 0.2691 - acc: 0.88 - ETA: 17s - loss: 0.2693 - acc: 0.88 - ETA: 17s - loss: 0.2705 - acc: 0.88 - ETA: 16s - loss: 0.2693 - acc: 0.88 - ETA: 16s - loss: 0.2679 - acc: 0.89 - ETA: 16s - loss: 0.2682 - acc: 0.89 - ETA: 15s - loss: 0.2749 - acc: 0.88 - ETA: 15s - loss: 0.2764 - acc: 0.88 - ETA: 

8792/8792 [==============================] - ETA: 28s - loss: 0.2658 - acc: 0.88 - ETA: 27s - loss: 0.2662 - acc: 0.88 - ETA: 25s - loss: 0.2465 - acc: 0.89 - ETA: 24s - loss: 0.2500 - acc: 0.90 - ETA: 24s - loss: 0.2447 - acc: 0.89 - ETA: 23s - loss: 0.2640 - acc: 0.89 - ETA: 23s - loss: 0.2789 - acc: 0.88 - ETA: 22s - loss: 0.2825 - acc: 0.88 - ETA: 22s - loss: 0.2887 - acc: 0.88 - ETA: 22s - loss: 0.2876 - acc: 0.88 - ETA: 21s - loss: 0.2884 - acc: 0.88 - ETA: 21s - loss: 0.2897 - acc: 0.87 - ETA: 20s - loss: 0.2850 - acc: 0.88 - ETA: 20s - loss: 0.2970 - acc: 0.87 - ETA: 20s - loss: 0.2999 - acc: 0.87 - ETA: 19s - loss: 0.2974 - acc: 0.87 - ETA: 19s - loss: 0.3003 - acc: 0.87 - ETA: 19s - loss: 0.2958 - acc: 0.87 - ETA: 19s - loss: 0.2930 - acc: 0.87 - ETA: 19s - loss: 0.2896 - acc: 0.87 - ETA: 18s - loss: 0.2887 - acc: 0.88 - ETA: 18s - loss: 0.2916 - acc: 0.87 - ETA: 17s - loss: 0.2958 - acc: 0.87 - ETA: 17s - loss: 0.2966 - acc: 0.87 - ETA: 17s - loss: 0.2977 - acc: 0.88 - ETA: 

8792/8792 [==============================] - ETA: 28s - loss: 0.3325 - acc: 0.87 - ETA: 28s - loss: 0.2894 - acc: 0.88 - ETA: 28s - loss: 0.3250 - acc: 0.87 - ETA: 28s - loss: 0.3041 - acc: 0.88 - ETA: 28s - loss: 0.2955 - acc: 0.88 - ETA: 30s - loss: 0.3097 - acc: 0.87 - ETA: 31s - loss: 0.3134 - acc: 0.87 - ETA: 31s - loss: 0.3235 - acc: 0.86 - ETA: 30s - loss: 0.3140 - acc: 0.87 - ETA: 29s - loss: 0.3107 - acc: 0.87 - ETA: 28s - loss: 0.3094 - acc: 0.87 - ETA: 27s - loss: 0.3092 - acc: 0.87 - ETA: 26s - loss: 0.3056 - acc: 0.87 - ETA: 26s - loss: 0.3061 - acc: 0.87 - ETA: 26s - loss: 0.2998 - acc: 0.87 - ETA: 25s - loss: 0.2955 - acc: 0.87 - ETA: 25s - loss: 0.2941 - acc: 0.87 - ETA: 24s - loss: 0.2923 - acc: 0.87 - ETA: 23s - loss: 0.2939 - acc: 0.87 - ETA: 22s - loss: 0.2926 - acc: 0.87 - ETA: 22s - loss: 0.2930 - acc: 0.87 - ETA: 21s - loss: 0.2915 - acc: 0.87 - ETA: 20s - loss: 0.2877 - acc: 0.87 - ETA: 20s - loss: 0.2856 - acc: 0.87 - ETA: 19s - loss: 0.2852 - acc: 0.87 - ETA: 